In [51]:
%matplotlib inline

# 计算给定数据集的Shannon熵
$$shannonEnt = -\sum_{i=1}^n p(x_i) \log_2p(x_i) $$

In [1]:
def calcShannonEnt(dataset):
    numEntris = len(dataset)
    labelCounts = {}
    for featVec in dataset:
        currentLabel = featVec[-1] #每行数据中的最后一个数，即数据的决策结果
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel]+=1  #labelCounts记录了类的种类（keys）和每个类的数量（values）
    #
    shannonEnt = 0
    for key in labelCounts:
        prob = float(labelCounts[key])/numEntris
        shannonEnt -= prob*log(prob,2)
    return shannonEnt

# 创造示例数据

In [2]:
def createDataSet1():    # 创造示例数据
    dataSet = [['Long', 'Coarse', 'Male'],
               ['Short', 'Coarse', 'Male'],
               ['Short', 'Coarse', 'Male'],
               ['Long', 'Thin', 'Female'],
               ['Short', 'Thin', 'Female'],
               ['Short', 'Coarse', 'Female'],
               ['Long', 'Thin', 'Female']]
    labels = ['Hair','Vioce']  #两个特征
    return dataSet,labels

###### 按某个特征分类后的数据，如特征为Long的数据：[['Coarse', 'Male'], ['Thin', 'Female'], ['Coarse', 'Female'], ['Coarse', 'Female']]

In [3]:
def splitDataSet(dataSet,axis,value):  #axis = [0, ..., numFeature]  value = 某一特征的所有取值
    retDataSet = []
    for featVec in dataSet:
        if featVec[axis] == value:
            reduceFeatVec = featVec[:axis]
            reduceFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reduceFeatVec)
    return retDataSet

# 选择最优分类特征

In [4]:
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0])-1
    baseEntropy = calcShannonEnt(dataSet) #原始数据的熵
    bestInfoGain = 0
    bestFfeature = -1
    for i in range(numFeatures):   #循环所有特征
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList)  #某个特征的取值，如[long,short]
        newEntropy = 0
        for value in uniqueVals:    
            subDataSet = splitDataSet(dataSet,i,value) #按某一特征的取值分类，如Long
            prob = len(subDataSet)/float(len(dataSet))
            newEntropy += prob*calcShannonEnt(subDataSet)  #计算按该特征分类的熵，如DATASET(LONG)和DATASET（Short）的熵
        infoGain = baseEntropy - newEntropy   #计算增益，原始熵-Dataset(long)的熵-Dataset(short)的熵
        if (infoGain>bestInfoGain):
            bestInfoGain = infoGain
            bestFfeature = i   #选出最优分类特征
    return bestFfeature

In [5]:
def majorityCnt(classList):    #按分类后类别数量排序，比如：最后分类为2男1女，则判定为男；
    classCount={}
    for vote in classList:
        if vote not in classCount.keys():
            classCount[vote]=0
        classCount[vote]+=1
    sortedClassCount = sorted(classCount.items(),key=operator.itemgetter(1),reverse=True)
    return sortedClassCount[0][0]

In [18]:
def createTree(dataSet,labels):
    classList=[example[-1] for example in dataSet]  # 类别：男或女
    if classList.count(classList[0])==len(classList):
        return classList[0]
    if len(dataSet[0])==1:
        return majorityCnt(classList)
    bestFeat=chooseBestFeatureToSplit(dataSet) #选择最优特征
    bestFeatLabel=labels[bestFeat]
    print bestFeat, bestFeatLabel
    myTree={bestFeatLabel:{}} #分类结果以字典形式保存
    del(labels[bestFeat])
    featValues=[example[bestFeat] for example in dataSet]
    print featValues
    uniqueVals=set(featValues)
    for value in uniqueVals:
        subLabels=labels[:]
        print "d%s" %subLabels
        myTree[bestFeatLabel][value]=createTree(splitDataSet\
                            (dataSet,bestFeat,value),subLabels)
    return myTree

In [19]:
from math import log
import operator
if __name__=='__main__':
    dataSet, labels=createDataSet1()  # 创造示列数据
    createTree(dataSet, labels)  # 输出决策树模型结果

1 Vioce
['Coarse', 'Coarse', 'Coarse', 'Thin', 'Thin', 'Coarse', 'Thin']
d['Hair']
0 Hair
['Long', 'Short', 'Short', 'Short']
d[]
d[]
d['Hair']
